<a href="https://colab.research.google.com/github/EricaAndreose/enoam_doc/blob/main/Generate_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import rdflib
from datetime import datetime
from rdflib import URIRef, Namespace, Literal
from rdflib.namespace import RDF, RDFS, DCTERMS, FOAF, XSD, SKOS, OWL
from lib.location3 import get_coordinates, normalize_location
import os

Importazione librerie:

1. **rdflib**: libreria Python per lavorare con grafi RDF.
2. **datetime**: modulo Python utilizzato per lavorare con date e orari.
3. **rdflib** (URIRef, rdflib.Namespace, rdflib.Literal): specifici costrutti di RDFLib per lavorare rispettivamente con URI, namespace e valori letterali.
4. **rdflib.namespace.RDF, rdflib.namespace.RDFS, rdflib.namespace.DCTERMS, rdflib.namespace.FOAF, rdflib.namespace.XSD, rdflib.namespace.SKOS, rdflib.namespace.OWL**: namespace standard per l'RDF, ciascuno utilizzato per rappresentare diversi tipi di dati e metadati. Vediamoli nel dettaglio:
  * **RDF**: RDF Schema, utilizzato per descrivere risorse e le loro proprietà.

  * **RDFS**: estensione di RDF per fornire vocabolari più ricchi.
  * **DCTERMS**: Dublin Core Terms, un set di termini utilizzati per descrivere le risorse.
  * **FOAF**: Friend of a Friend, un vocabolario RDF per descrivere persone, le loro relazioni e i loro luoghi.
  * **XSD**: XML Schema Definition, usato per definire tipi di dati.
  * **SKOS**: Simple Knowledge Organization System, utilizzato per la rappresentazione dei thesauri.
  * **OWL**: Web Ontology Language, utilizzato per definire ontologie.

5. **os**: modulo standard di Python utilizzato per interagire con il sistema operativo, ad esempio per lavorare con file, directory e variabili d'ambiente.

Vengono inoltre importate funzioni da un modulo locale:

`get_coordinates` e `normalize_location` sono due funzioni importate dal modulo **lib.location3**.

In [ ]:
# ADD MAIN FRBR STATEMENTS (FRBR entities, relations between them, labels)
def add_frbr(g, document, work, expression, manifestation, fabio, frbr):
    g.add((work, RDF.type, fabio.Work))
    g.add((expression, RDF.type, fabio.Expression))
    g.add((manifestation, RDF.type, fabio.Manifestation))
    g.add((work, RDFS.label, Literal(f'{document["title"]} (FRBR Work)', datatype=XSD.string)))
    g.add((work, frbr.realization, expression))
    g.add((work, fabio.hasManifestation, manifestation))
    g.add((expression, frbr.embodiment, manifestation))
    g.add((expression, RDFS.label, Literal(f'{document["title"]} (FRBR Expression)', datatype=XSD.string)))
    g.add((manifestation, RDFS.label, Literal(f'{document["title"]} (FRBR Manifestation)', datatype=XSD.string)))


La funzione `add_frbr` aggiunge triple RDF al grafo g per rappresentare entità e relazioni basate sul modello FRBR (Functional Requirements for Bibliographic Records).

Questo modello è utilizzato per descrivere la struttura concettuale di opere bibliografiche, e la funzione in questione si occupa di creare e collegare le entità principali del modello: Work, Expression e Manifestation.

La funzione prende in input:

- **g**: un oggetto Graph di RDFLib che rappresenta il grafo RDF in cui verranno aggiunte le dichiarazioni.

- **document**: metadati contenuti su MongoDB in formato dizionario.

- **work, expression, manifestation**: URIRef che rappresentano le entità Work, Expression e Manifestation nel grafo RDF.

- **fabio**: namespace che contiene classi e proprietà del modello FaBiO (FRBR-aligned Bibliographic Ontology), utilizzato per estendere FRBR.

- **frbr**: namespace che contiene le relazioni standard del modello FRBR.

La funzione aggiunge al grafo `g` le triplette che definiscono le tipologie RDF delle entità:

- **work** come un'istanza della classe `fabio.Work`.

- **expression** come un'istanza della classe `fabio.Expression`.

- **manifestation** come un'istanza della classe `fabio.Manifestation`.

Ogni entità (work, expression, manifestation) riceve un'etichetta (**RDFS.label**) che descrive il tipo di entità e include il titolo del documento.

Creazione delle relazioni tra le entità:

**work** è collegato a **expression** tramite la proprietà `frbr.realization`, indicando che expression è una realizzazione di work.

**work** è collegato a **manifestation** tramite la proprietà `fabio.hasManifestation`, indicando che manifestation è una manifestazione di work.

**expression** è collegato a **manifestation** tramite la proprietà `frbr.embodiment`, indicando che manifestation è l'embodiment di expression.



In [ ]:
# ADD IDENTIFIER AS DOI
def add_ident(g, value, manifestation):
    g.add((manifestation, DCTERMS.identifier, Literal(f'10.48678/unibo/aldomoro{".".join(value.split("_"))}', datatype=XSD.string)))


# ADD TITLE
def add_title(g, value, work):
    g.add((work, DCTERMS.title, Literal(value, datatype=XSD.string)))


# ADD CREATOR
def add_creator(g, work, moro):
    g.add((work, DCTERMS.creator, moro))


# ADD CURATOR
def add_contributor(g, value, expression, dataset_ns):
    curator = URIRef(f'{dataset_ns}{value.replace(" ", "-").lower()}')
    g.add((curator, RDF.type, FOAF.Person))
    g.add((curator, RDFS.label, Literal(value, datatype=XSD.string)))
    g.add((expression, DCTERMS.contributor, curator))


# ADD ABSTRACT
def add_abstract(g, value, expression):
    g.add((expression, DCTERMS.abstract, Literal(value, datatype=XSD.string)))

Descriviamo queste funzioni:

1. `add_ident`:aggiunge un identificatore (in questo caso un DOI) all'entità manifestation nel grafo RDF. Il DOI viene creato concatenando il prefisso 10.48678/unibo/aldomoro con la stringa derivata da `value` (input del metadato `ident` di MongoDB), dove gli underscore (_) vengono sostituiti da punti (.).
La tripla risultante associa il DOI alla manifestation tramite la proprietà `DCTERMS.identifier`.

2. `add_title`:aggiunge un titolo all'entità work nel grafo RDF. La tripla risultante associa il titolo (sempre derivante da MongoDB) all'entità work utilizzando la proprietà `DCTERMS.title`.

3. `add_creator`:associa un creatore (autore) all'entità work nel grafo RDF. La tripla risultante associa l'autore (Aldo Moro risulta sempre come autore) all'entità work tramite la proprietà `DCTERMS.creator`.

4. `add_contributor`:aggiunge un curatore come contributore all'entità expression nel grafo RDF. Viene creato un URI per il curatore basato sullo spazio dei nomi `dataset_ns` e sul valore normalizzato di `value` (convertendo gli spazi in trattini e passando tutto in minuscolo). Il curatore viene dichiarato come un'istanza di `FOAF.Person`, e viene anche aggiunta un'etichetta (label) con il nome del curatore come Literal. Infine, la tripla risultante associa il curatore all'entità expression tramite la proprietà `DCTERMS.contributor`.

5. `add_abstract`:aggiunge un abstract all'entità expression nel grafo RDF. La tripla risultante associa l'abstract (sempre derivante dai metadati di MongoDB) all'entità expression tramite la proprietà` DCTERMS.abstract`.